# Tracking APIs ( Built-in with OpenCV)

In [1]:
import cv2

In [2]:
def ask_for_tracker():
    print("Welcome! What Tracker API would you like to use?")
    print("Enter 0 for BOOSTING: ")
    print("Enter 1 for MIL: ")
    print("Enter 2 for KCF: ")
    print("Enter 3 for TLD: ")
    print("Enter 4 for MEDIANFLOW: ")
    choice = input("Please select your tracker: ")
    
    if choice == '0':
        tracker = cv2.TrackerBoosting_create()
    if choice == '1':
        tracker = cv2.TrackerMIL_create()
    if choice == '2':
        tracker = cv2.TrackerKCF_create()
    if choice == '3':
        tracker = cv2.TrackerTLD_create()
    if choice == '4':
        tracker = cv2.TrackerMedianFlow_create()


    return tracker
 

In [3]:
tracker = ask_for_tracker()

Welcome! What Tracker API would you like to use?
Enter 0 for BOOSTING: 
Enter 1 for MIL: 
Enter 2 for KCF: 
Enter 3 for TLD: 
Enter 4 for MEDIANFLOW: 


Please select your tracker:  1


In [4]:
tracker

<TrackerMIL 000001938307B5F0>

In [5]:
str(tracker).split()[0][1:]

'TrackerMIL'

In [6]:
# str(tracker).split()[0][1:]

In [11]:
tracker = ask_for_tracker()
tracker_name = str(tracker).split()[0][1:]

# Read video
cap = cv2.VideoCapture(0)

# Read first frame.
ret, frame = cap.read()


# Special function allows us to draw on the very first frame our desired ROI
roi = cv2.selectROI(frame, False)

# Initialize tracker with first frame and bounding box
ret = tracker.init(frame, roi)

while True:
    # Read a new frame
    ret, frame = cap.read()
    
    
    # Update tracker
    success, roi = tracker.update(frame)
    
    # roi variable is a tuple of 4 floats
    # We need each value and we need them as integers
    (x,y,w,h) = tuple(map(int,roi))
    
    # Draw Rectangle as Tracker moves
    if success:
        # Tracking success
        p1 = (x, y)
        p2 = (x+w, y+h)
        cv2.rectangle(frame, p1, p2, (0,255,0), 3)
    else :
        # Tracking failure
        cv2.putText(frame, "Failure to Detect Tracking!!", (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),3)

    # Display tracker type on frame
    cv2.putText(frame, tracker_name, (20,400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),3);

    # Display result
    cv2.imshow(tracker_name, frame)

    # Exit if ESC pressed
    k = cv2.waitKey(1) & 0xff
    if k == 27 : 
        break
        
cap.release()
cv2.destroyAllWindows()

Welcome! What Tracker API would you like to use?
Enter 0 for BOOSTING: 
Enter 1 for MIL: 
Enter 2 for KCF: 
Enter 3 for TLD: 
Enter 4 for MEDIANFLOW: 


Please select your tracker:  2


In [17]:
import cv2
from random import randint, shuffle
import numpy as np
 
trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT'] 

# Set video to load

 
# Create a video capture object to read videos
cap = cv2.VideoCapture("../DATA/full_bodies.avi")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('people.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))  

# Read first frame
success, frame = cap.read()
# quit if unable to read the video file
if not success:
    print('Failed to read video')
    sys.exit(1)

  
    
## Select boxes
bboxes = []
colors = [] 
 
# OpenCV's selectROI function doesn't work for selecting multiple objects in Python
# So we will call this function in a loop till we are done selecting all objects
# while True:
#     # draw bounding boxes over objects
#     # selectROI's default behaviour is to draw box starting from the center
#     # when fromCenter is set to false, you can draw box starting from top left corner
#     bbox = cv2.selectROI('MultiTracker', frame)
#     bboxes.append(bbox)
#     colors.append((randint(0, 255), randint(0, 255), randint(0, 255)))
#     print("Press q to quit selecting boxes and start tracking")
#     print("Press any other key to select next object")
#     k = cv2.waitKey(0) & 0xFF
#     if (k == 113):  # q is pressed
#     break
    
    
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
hog = cv2.HOGDescriptor()#'../DATA/DATA/haarcascades/hogcascade_pedestrians.xml'
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
#rects, weights = hog.detectMultiScale(gray, winStride=(4, 4), padding=(4, 4), scale=1.02)
rects, weights = hog.detectMultiScale(gray, winStride=(2, 2), padding=(10, 10), scale=1.02)

#shuffle(rects)
#print(rects)

#rects = list(rects)
#rects.sort(key=lambda rec: rec[2] * rec[3])
#rects = np.array(rects)

#print(rects)

prev = [[],[],[],[],[],[]]

for i, rec in enumerate(rects):
    if len(bboxes) == 5:
        break
    if(weights[i] >= 0.8 and tuple(map(int, rec)) not in bboxes):
        bboxes.append(tuple(map(int, rec)))
        prev[i-1].append((int(rec[0] + rec[2]/2), int(rec[1] + rec[3]/2)))
        colors.append((0, int(255 * weights[i]), 0)) #randint(0, 255)
    
print('Selected bounding boxes {}'.format(bboxes))

# Specify the tracker type
trackerType = "BOOSTING"   
 
# Create MultiTracker object
multiTracker = cv2.MultiTracker_create()
 
# Initialize MultiTracker
for bbox in bboxes:
    multiTracker.add(createTrackerByName(trackerType), frame, bbox)

# Process video and track objects
while cap.isOpened():
    success, frame = cap.read()
    
    if not success:
        break

    # get updated location of objects in subsequent frames
    success, boxes = multiTracker.update(frame)

    # draw tracked objects
    for i, newbox in enumerate(boxes):
        p1 = (int(newbox[0]), int(newbox[1]))
        p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
        cv2.rectangle(frame, p1, p2, colors[i], 2, 1)
        cv2.circle(frame,(int(newbox[0] + newbox[2]/2), int(newbox[1] + newbox[3]/2)),8,(0,0,255),-1)
        
        prev[i].append((int(newbox[0] + newbox[2]/2), int(newbox[1] + newbox[3]/2)))
        
        for j in range(1,len(prev[i])):
            cv2.line(frame, prev[i][j-1], prev[i][j], (0,255,0), 3)
        
    # show frame
    out.write(frame)
    width = int(frame.shape[1] )
    height = int(frame.shape[0])
    frame = cv2.resize(frame, (width, height))
    cv2.imshow('MultiTracker', frame)

    # quit on ESC button
    k = cv2.waitKey(1) & 0xff
    if k == 27 : 
        break
        
cap.release()
out.release()
cv2.destroyAllWindows()


Selected bounding boxes [(571, 14, 184, 368), (621, 157, 97, 193), (232, 189, 73, 145)]
